In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Set random seed for reproducibility
tf.random.set_seed(42)


In [3]:
# Hyperparameters
image_size = (224, 224)  # Input size for MobileNetV2
batch_size = 32
epochs = 10
learning_rate = 0.0001
num_keypoints = 17  # Number of keypoints for pose estimation

In [4]:
# Load pre-trained MobileNetV2 as the backbone
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(224, 224, 3)
)

In [5]:
# Fine-tuning: Unfreeze the last few layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

In [6]:
# Add custom regression head for keypoints
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(num_keypoints * 2, activation="linear")(x)  # 2D keypoints

In [8]:
model = Model(inputs=base_model.input, outputs=predictions)


In [9]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())

In [10]:
# Data augmentation for pose estimation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [11]:

train_data_dir = r'C:/Users/5A_Traders/Downloads/FYP_ON_DEV/FYP_IntelliTrain/Datasets/pose/LSP/images'
test_data_dir = r'C:/Users/5A_Traders/Downloads/FYP_ON_DEV/FYP_IntelliTrain/Datasets/pose/LSP/test'

In [12]:
# Placeholder for training and validation datasets (to be replaced with pose datasets)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None
)

Found 1936 images belonging to 1 classes.


In [13]:
val_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None
)

Found 64 images belonging to 1 classes.


In [14]:
# Placeholder for ground truth keypoints (to be aligned with data generators)
# Assume `train_keypoints` and `val_keypoints` are numpy arrays of shape (num_samples, num_keypoints*2)
train_keypoints = None  # Load or preprocess ground truth keypoints for training
val_keypoints = None  # Load or preprocess ground truth keypoints for validation

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=1
)

In [ ]:
model.save("pose_estimation_model.h5")